In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import cleaning
from datetime import timedelta
%load_ext autoreload
%autoreload 2

path = "../data/mimic-iv-clinical-database-demo-2.2/hosp/"
path2 = "../data/mimic-iv-clinical-database-demo-2.2/icu/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
## reading files and creating dataframes for tables
files = ["admissions", "patients", "labevents", "d_labitems", "prescriptions","pharmacy","transfers", "diagnoses_icd","d_icd_diagnoses","procedures_icd","d_icd_procedures"]
dfs = {}

for name in files:
    dfs[name] = pd.read_csv(path + f"{name}.csv.gz")

admissions = dfs["admissions"]
patients = dfs["patients"]
labs = dfs["labevents"]
d_labitems = dfs["d_labitems"]
prescriptions = dfs["prescriptions"]
pharmacy = dfs["pharmacy"]
transfers=dfs["transfers"]
diagnoses = dfs["diagnoses_icd"]
d_diagnoses = dfs["d_icd_diagnoses"]
procedures = dfs["procedures_icd"]
d_procedures = dfs["d_icd_procedures"]

files2 = ["icustays", "inputevents", "outputevents", "procedureevents","chartevents", "datetimeevents", "d_items"]
dfs2 = {}

for name in files2:
    dfs2[name] = pd.read_csv(path2 + f"{name}.csv.gz")
icustays = dfs2["icustays"]
inputevents = dfs2["inputevents"]
outputevents = dfs2["outputevents"]
procedureevents = dfs2["procedureevents"]
chartevents = dfs2["chartevents"]
datetimeevents = dfs2["datetimeevents"]
d_items = dfs2["d_items"]

In [30]:
test = cleaning.get_base("Urinary Tract Infection")
test.head()

,hadm_id,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,...,edouttime,hospital_expire_flag,stay_id,first_careunit,last_careunit,intime,outtime,los,ICU_length,Hospital_length
0,23473524,10002428,F,80,2155,2011 - 2013,NaN,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,...,2156-05-11 16:53:00,0,35479615.0,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,10 days 23:27:12,10 days 23:27:00
1,25797028,10002428,F,80,2155,2011 - 2013,NaN,2155-07-14 19:15:00,2155-07-15 18:37:00,NaN,...,2155-07-14 20:04:00,0,NaN,NaN,NaN,NaT,NaT,NaN,NaT,0 days 23:22:00
2,25177949,10032725,F,38,2143,2011 - 2013,2143-03-30,2143-02-17 14:20:00,2143-03-16 17:15:00,NaN,...,2143-02-17 15:18:00,0,NaN,NaN,NaN,NaT,NaT,NaN,NaT,27 days 02:55:00
3,22490490,10037928,F,78,2175,2011 - 2013,NaN,2177-07-14 16:55:00,2177-07-24 13:33:00,NaN,...,2177-07-14 20:38:00,0,31552399.0,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2177-07-14 20:38:00,2177-07-15 16:08:36,0.812917,0 days 19:30:36,9 days 20:38:00
4,29802992,10037928,F,78,2175,2011 - 2013,NaN,2179-07-25 00:06:00,2179-07-28 15:54:00,NaN,...,2179-07-25 01:17:00,0,NaN,NaN,NaN,NaT,NaT,NaN,NaT,3 days 15:48:00


In [31]:
print("Unique subject_id:", test['subject_id'].nunique())
print("Unique hadm_id:", test['hadm_id'].nunique())
print("Unique stay_id:", test['stay_id'].nunique())
print("Rows with no ICU stay:", test['stay_id'].isna().sum())
icu_counts = test.groupby('hadm_id')['stay_id'].nunique()
multiple_icu = icu_counts[icu_counts > 1]
print("Admissions with >1 ICU stay:\n", multiple_icu)

Unique subject_id: 22
Unique hadm_id: 36
Unique stay_id: 22
Rows with no ICU stay: 17
Admissions with >1 ICU stay:
 hadm_id
27417763    3
28477280    2
Name: stay_id, dtype: int64


In [32]:
vital_keywords = ["sodium", "potassium", "bun", "urea", "creatinine", "glucose", "ph", "lactate",
    "platelet", "wbc", "white blood", "hemoglobin", "hgb",
    "ast", "alt", "bilirubin", "inr"]

vitals_lookup = d_items[
    d_items["label"].str.contains('|'.join(vital_keywords), case=False, na=False)
][["itemid", "label", "unitname"]].sort_values("label")

vitals_lookup

,itemid,label,unitname
1759,228007,14 G Phlebitis Scale,NaN
1782,228009,16 G Phlebitis Scale,NaN
1780,228011,18 G Phlebitis Scale,NaN
1773,228013,20 G Phlebitis Scale,NaN
1777,228015,22 G Phlebitis Scale,NaN
...,...,...,...
1979,225676,ZPhenobarbital,NaN
1949,225645,ZPhenytoin (Dilantin),NaN
1931,225678,ZPlatelet Count,NaN
1959,220640,ZPotassium (serum),NaN


In [33]:
vitals = cleaning.get_vitals(test,4,4)
vitals.head()

,hadm_id,heart_rate_max,blood_pressure_min
0,23473524,73.0,NaN
1,25797028,NaN,NaN
2,25177949,NaN,NaN
3,22490490,94.0,62.0
4,29802992,NaN,NaN


In [34]:
columns = ['hadm_id','subject_id','stay_id',
    'anchor_age',
    'gender',
    'race',
    'admission_type',
    'admission_location',
    'admittime',
    'dischtime',
    'hospital_expire_flag',
    'intime',
    'outtime',
    'ICU_length',
    'Hospital_length',
]
cleaned = test[columns]
cleaned.head()

,hadm_id,subject_id,stay_id,anchor_age,gender,race,admission_type,admission_location,admittime,dischtime,hospital_expire_flag,intime,outtime,ICU_length,Hospital_length
0,23473524,10002428,35479615.0,80,F,WHITE,EW EMER.,EMERGENCY ROOM,2156-05-11 14:49:00,2156-05-22 14:16:00,0,2156-05-11 14:49:34,2156-05-22 14:16:46,10 days 23:27:12,10 days 23:27:00
1,25797028,10002428,NaN,80,F,WHITE,EU OBSERVATION,EMERGENCY ROOM,2155-07-14 19:15:00,2155-07-15 18:37:00,0,NaT,NaT,NaT,0 days 23:22:00
2,25177949,10032725,NaN,38,F,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,2143-02-17 14:20:00,2143-03-16 17:15:00,0,NaT,NaT,NaT,27 days 02:55:00
3,22490490,10037928,31552399.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2177-07-14 16:55:00,2177-07-24 13:33:00,0,2177-07-14 20:38:00,2177-07-15 16:08:36,0 days 19:30:36,9 days 20:38:00
4,29802992,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2179-07-25 00:06:00,2179-07-28 15:54:00,0,NaT,NaT,NaT,3 days 15:48:00


In [35]:
cleaned = cleaning.get_max_creatinine_bun(cleaned)
cleaned.head()

,hadm_id,subject_id,stay_id,anchor_age,gender,race,admission_type,admission_location,admittime,dischtime,hospital_expire_flag,intime,outtime,ICU_length,Hospital_length,creatinine_admission_max,bun_admission_max
0,23473524,10002428,35479615.0,80,F,WHITE,EW EMER.,EMERGENCY ROOM,2156-05-11 14:49:00,2156-05-22 14:16:00,0,2156-05-11 14:49:34,2156-05-22 14:16:46,10 days 23:27:12,10 days 23:27:00,0.4,12.0
1,25797028,10002428,NaN,80,F,WHITE,EU OBSERVATION,EMERGENCY ROOM,2155-07-14 19:15:00,2155-07-15 18:37:00,0,NaT,NaT,NaT,0 days 23:22:00,0.8,12.0
2,25177949,10032725,NaN,38,F,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,2143-02-17 14:20:00,2143-03-16 17:15:00,0,NaT,NaT,NaT,27 days 02:55:00,0.9,21.0
3,22490490,10037928,31552399.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2177-07-14 16:55:00,2177-07-24 13:33:00,0,2177-07-14 20:38:00,2177-07-15 16:08:36,0 days 19:30:36,9 days 20:38:00,1.4,28.0
4,29802992,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2179-07-25 00:06:00,2179-07-28 15:54:00,0,NaT,NaT,NaT,3 days 15:48:00,1.8,47.0


questions: what to do with hospitalizations that have multiple icu stays, or none? 

what to do with patients that have multiple hospitilizations? 

are the diagnosis tables binary?

should i be looking at labevents or chartevents for the vitals? which itemids should I use?

how to filter for antibiotics? what are Comorbidities?


In [36]:
pharmacy_uti = pharmacy[pharmacy["hadm_id"].isin(cleaned["hadm_id"])].copy()
medication_count= pharmacy_uti["medication"].value_counts()
#medication_count.to_csv("medication_count.csv")

In [39]:
procedures_uti = procedures[procedures["hadm_id"].isin(cleaned["hadm_id"])].copy()
procedures_uti

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
13,10007818,22987108,15,2146-06-20,02H633Z,10
14,10007818,22987108,13,2146-06-22,0BH17EZ,10
15,10007818,22987108,17,2146-06-21,B211YZZ,10
16,10007818,22987108,11,2146-06-11,3E0G76Z,10
17,10007818,22987108,6,2146-06-29,0BCH8ZZ,10
...,...,...,...,...,...,...
620,10016742,29281842,1,2178-07-03,5A1955Z,10
697,10002428,23473524,1,2156-05-11,9672,9
698,10002428,23473524,2,2156-05-11,9604,9
699,10002428,23473524,3,2156-05-11,3891,9


In [42]:
procedures_uti = procedures_uti.merge(d_procedures,on=["icd_code","icd_version"], how="left")

In [46]:
vent_keywords = [
    "ventilation", "endotracheal", "intubation", "mechanical ventilation"
]

vent_mask = procedures_uti['long_title'].str.contains('|'.join(vent_keywords), case=False, na=False)

vent_procs = procedures_uti[vent_mask]
vent_procs_hadm = vent_procs["hadm_id"]

In [48]:
cleaned['vent_or_intubation'] = cleaned['hadm_id'].isin(vent_procs_hadm).astype(int)
cleaned

,hadm_id,subject_id,stay_id,anchor_age,gender,race,admission_type,admission_location,admittime,dischtime,hospital_expire_flag,intime,outtime,ICU_length,Hospital_length,creatinine_admission_max,bun_admission_max,vent_or_intubation
0,23473524,10002428,35479615.0,80,F,WHITE,EW EMER.,EMERGENCY ROOM,2156-05-11 14:49:00,2156-05-22 14:16:00,0,2156-05-11 14:49:34,2156-05-22 14:16:46,10 days 23:27:12,10 days 23:27:00,0.4,12.0,1
1,25797028,10002428,NaN,80,F,WHITE,EU OBSERVATION,EMERGENCY ROOM,2155-07-14 19:15:00,2155-07-15 18:37:00,0,NaT,NaT,NaT,0 days 23:22:00,0.8,12.0,0
2,25177949,10032725,NaN,38,F,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,2143-02-17 14:20:00,2143-03-16 17:15:00,0,NaT,NaT,NaT,27 days 02:55:00,0.9,21.0,0
3,22490490,10037928,31552399.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2177-07-14 16:55:00,2177-07-24 13:33:00,0,2177-07-14 20:38:00,2177-07-15 16:08:36,0 days 19:30:36,9 days 20:38:00,1.4,28.0,0
4,29802992,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2179-07-25 00:06:00,2179-07-28 15:54:00,0,NaT,NaT,NaT,3 days 15:48:00,1.8,47.0,0
5,24225421,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-09-28 23:05:00,2178-10-02 17:13:00,0,NaT,NaT,NaT,3 days 18:08:00,1.4,26.0,0
6,24656677,10037928,39804682.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-12-21 05:30:00,2178-12-26 18:35:00,0,2178-12-21 06:05:18,2178-12-22 02:16:08,0 days 20:10:50,5 days 13:05:00,1.3,20.0,0
7,22228639,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EU OBSERVATION,PHYSICIAN REFERRAL,2183-08-04 04:04:00,2183-08-04 16:07:00,0,NaT,NaT,NaT,0 days 12:03:00,1.9,26.0,0
8,20291550,10008454,31959184.0,26,F,WHITE,EW EMER.,EMERGENCY ROOM,2110-11-30 06:31:00,2110-12-10 15:53:00,0,2110-11-30 17:11:36,2110-12-05 16:48:24,4 days 23:36:48,10 days 09:22:00,0.7,11.0,0
9,27984218,10020640,30849778.0,91,F,WHITE,EW EMER.,EMERGENCY ROOM,2153-02-13 00:22:00,2153-02-20 13:52:00,0,2153-02-13 01:38:00,2153-02-15 15:00:57,2 days 13:22:57,7 days 13:30:00,0.8,33.0,0


In [49]:
cleaned['vent_or_intubation'].value_counts()

vent_or_intubation
0    31
1     8
Name: count, dtype: int64